Practica 1 FAA
======


#### Lucía Asencio y Alfonso Carvajal
#### Grupo 1461, pareja 16


## Particionado
----------------

Para esta entrega hemos implementado dos estrategias de particionado, 
* Valicación __simple,__ y
* Validación __cruzada__


### Validación simple

En el primer caso, el usuario escoge qué porcentaje de los datos va a ser usado para probar el modelo (el conjunto de test) y los datos restantes serán usados para entrenar el modelo. Esta validación conlleva por tanto una única partición de los datos.

Para una consulta rápida, adjuntamos el código que crea las particiones en validación simple.

``` python
    #Crea particiones segun el metodo tradicional de division de los datos segun el porcentaje deseado.
    # Devuelve una lista de particiones (clase Particion)
    def creaParticiones(self,datos,seed=None):
        # we assign a new value to seed only if it is None already
        if seed == None:
            random.seed(seed)

        # number of rows of the datos Matrix (number of inidvidual data)
        totalRows = len(datos)
        # Assuming porcentajeDeseado refers to the percentage of the data we want to save for testing
        # we obtain the number of rows (data) to use for testing
        testRows = int( (self.porcentajeDeseado * totalRows)/100 )
        rows = list(range(0, totalRows))
        random.shuffle(rows)
        particionSimple = Particion()
        # array size of totalRows - testRows
        particionSimple.indicesTrain = rows[testRows :]
        # array size of testRows
        particionSimple.indicesTest = rows[: testRows]

        self.listaParticiones.append(particionSimple)


        return self.listaParticiones```


#### Lenses
Comencemos viendo cómo la validación simple particiona los datos de las lentillas proporcionados para la práctica. A través del parámetro `porcentaje`, configuramos el porcentaje de datos que van destinados al conjunto de prueba. Consultamos en el objeto estrategia el número y estructura de la/s particiones creadas.

In [ ]:


from plantillasArquitectura import Datos as d
from plantillasArquitectura import EstrategiaParticionado as ep

porcentaje = 25
dataset = d.Datos('./plantillasArquitectura/datos/conjunto_datos_lentillas.txt')
estrategia = ep.ValidacionSimple(porcentaje)
listaParticiones = estrategia.creaParticiones(dataset.datos)

print('Tal y como esperamos, la lista de particiones de la estrategia tiene longitud ', len(listaParticiones))
print('\nHay ', len(listaParticiones[0].indicesTrain), 'índices (más o menos el ',100-porcentaje,'% de ',len(dataset.datos),', que es el número de datos) destinados al entrenamiento del modelo, y son:')
print(listaParticiones[0].indicesTrain)
print('\nLos ',len(listaParticiones[0].indicesTest),' índices restantes forman el conjunto de test: ')
print(listaParticiones[0].indicesTest)

#### Tic-tac-toe
Como en este fichero hay mayor número de datos, imprimiremos simplemente el número de particiones (que, al ser validación simple, esperamos que sea uno), y el número de elementos de cada conjunto de la partición, para comprobar que se obtiene el porcentaje deseado.

In [21]:
porcentaje = 30
dataset = d.Datos('./plantillasArquitectura/datos/tic-tac-toe.data')
estrategia = ep.ValidacionSimple(porcentaje)
listaParticiones = estrategia.creaParticiones(dataset.datos)

print('El fichero tic-tac-toe cuenta con un total de ', len(dataset.datos),' datos')
print('La lista de particiones de la estrategia tiene longitud', len(listaParticiones))
print('\nHay ', len(listaParticiones[0].indicesTrain), 'índices  en el conjunto de train, acorde con el ',100-porcentaje,'% esperado.')
print('Los ',len(listaParticiones[0].indicesTest),'índices restantes forman el conjunto de test, un',porcentaje ,'% de los datos totales')


El fichero tic-tac-toe cuenta con un total de  958  datos
La lista de particiones de la estrategia tiene longitud 1

Hay  671 índices  en el conjunto de train, acorde con el  70 % esperado.
Los  287 índices restantes forman el conjunto de test, un 30 % de los datos totales


#### German
Debido a la longitud de los datos, hacemos la simplificación anterior. Además, vamos a comprobar que ambos conjuntos (train y test) son disjuntos:

In [24]:
porcentaje = 20
dataset = d.Datos('./plantillasArquitectura/datos/german.data')
estrategia = ep.ValidacionSimple(porcentaje)
listaParticiones = estrategia.creaParticiones(dataset.datos)

print('El fichero cuenta con un total de ', len(dataset.datos),' datos')
print('La lista de particiones de la estrategia tiene longitud', len(listaParticiones))
print('\nHay ', len(listaParticiones[0].indicesTrain), 'índices  en el conjunto de train, acorde con el ',100-porcentaje,'% esperado.')
print('Los ',len(listaParticiones[0].indicesTest),'índices restantes forman el conjunto de test, un',porcentaje ,'% de los datos totales')
set1 = set(listaParticiones[0].indicesTest)
set2 = set(listaParticiones[0].indicesTrain)
set3 = set1.intersection(set2)
print('\nPor último, comprobamos el tamaño de la intersección de los conjuntos de train y test: ', len(set3) )

El fichero cuenta con un total de  1000  datos
La lista de particiones de la estrategia tiene longitud 1

Hay  800 índices  en el conjunto de train, acorde con el  80 % esperado.
Los  200 índices restantes forman el conjunto de test, un 20 % de los datos totales

Por último, comprobamos el tamaño de la intersección de los conjuntos de train y test:  0


### Validación cruzada
Si particionamos los datos usando validación cruzada, los datos se dividen en $K$ intervalos del mismo tamaño para evaluar el modelo. 

A lo largo de $K$ iteraciones, cada uno de estos K intervalos es usado una vez como conjunto de test, mientras los restantes $K-1$ conforman el conjunto de entrenamiento. Así, en cada iteración un error es calculado usando conjuntos de test y entrenamiento distintos, y el error final del modelo queda determinado por el error medio. En la implementación, este parámetro $K$ es elegido por el usuario.

Comprobaremos cómo funciona mediante algunos ejemplos, pero primero mostramos el código según el cual se crean las particiones de validación cruzada.

```python
def creaParticiones(self,datos,seed=None):
        # we assign a new value to seed only if it is None already
        if seed == None:
            random.seed(seed)

        # number of rows of the datos Matrix (number of inidvidual data)
        totalRows = len(datos)
        rows = list(range(0, totalRows))
        random.shuffle(rows)
        # number of rows in each partition (we take the integer part for the size of the test data)
        partitionSize = int(totalRows / self.numeroParticiones)
        for i in range(0, self.numeroParticiones):
            particionCruzada = Particion()
            test = rows[i * partitionSize : (i + 1) * partitionSize]
            train = list(set(rows) - set(test))
            particionCruzada.indicesTrain = train
            particionCruzada.indicesTest = test
            self.listaParticiones.append(particionCruzada)

        return self.listaParticiones
```

#### Lenses 


In [33]:
nParticiones = 5
dataset = d.Datos('./plantillasArquitectura/datos/conjunto_datos_lentillas.txt')
estrategia = ep.ValidacionCruzada(nParticiones)
listaParticiones = estrategia.creaParticiones(dataset.datos)

print('Contamos con ', len(dataset.datos), 'datos.')
print('La lista de particiones de la estrategia tiene longitud ', len(listaParticiones))
print('\nPara cada partición, vamos a imprimir el tamanio de los conjuntos de test y entrenamiento, y su suma.')
print('Esperariamos que cada tamanio fuera siempre el mismo, y que la suma fuera el numero total de datos')
for i in range(nParticiones):
    sizeTrain = len(listaParticiones[i].indicesTrain)
    sizeTest = len(listaParticiones[i].indicesTest)
    print('Particion', i+1, ': ', sizeTest,'+', sizeTrain, '=' , sizeTest+sizeTrain)

    
print('\nComo el conjunto de datos de lentillas es pequenio, vamos a imprimir los indices de test de cada particion, para ver que son realmente conjuntos distintos')
for i in range(nParticiones):
    print('El conjunto de test ', i+1, 'es', listaParticiones[i].indicesTest)

print('\nAdemas, vamos a comprobar que los conjuntos de test son disjuntos dos a dos, para ver que la particion tiene la estructura deseada:')
for i in range(nParticiones):
    for j in range(i+1, nParticiones):
        set1 = set(listaParticiones[i].indicesTest)
        set2 = set(listaParticiones[j].indicesTest)
        set3 = set1.intersection(set2)
        if(len(set3) == 0):
            print('La interseccion de los conjuntos de test', i+1, 'y', j+1, 'es vacia')


Contamos con  24 datos.
La lista de particiones de la estrategia tiene longitud  5

Para cada partición, vamos a imprimir el tamanio de los conjuntos de test y entrenamiento, y su suma.
Esperariamos que cada tamanio fuera siempre el mismo, y que la suma fuera el numero total de datos
Particion 1 :  4 + 20 = 24
Particion 2 :  4 + 20 = 24
Particion 3 :  4 + 20 = 24
Particion 4 :  4 + 20 = 24
Particion 5 :  4 + 20 = 24

Como el conjunto de datos de lentillas es pequenio, vamos a imprimir los indices de test de cada particion, para ver que son realmente conjuntos distintos
El conjunto de test  1 es [21, 5, 17, 16]
El conjunto de test  2 es [4, 18, 3, 1]
El conjunto de test  3 es [9, 13, 14, 0]
El conjunto de test  4 es [10, 7, 15, 12]
El conjunto de test  5 es [6, 19, 20, 23]

Ademas, vamos a comprobar que los conjuntos de test son disjuntos dos a dos, para ver que la particion tiene la estructura deseada:
La interseccion de los conjuntos de test 1 y 2 es vacia
La interseccion de los conju

####  German
Para este conjunto, como es más grande, nos limitaremos a mostrar el tamaño de cada conjunto de cada partición (al igual que en el caso anterior), y a comprobar que las intersecciones de las distintos conjuntos de test son nulas, ya que son demasiado largos para imprimirlos uno a uno.

In [35]:
nParticiones = 6
dataset = d.Datos('./plantillasArquitectura/datos/german.data')
estrategia = ep.ValidacionCruzada(nParticiones)
listaParticiones = estrategia.creaParticiones(dataset.datos)

print('Contamos con ', len(dataset.datos), 'datos.')
print('La lista de particiones de la estrategia tiene longitud ', len(listaParticiones))
print('\nPara cada partición, vamos a imprimir el tamanio de los conjuntos de test y entrenamiento, y su suma.')
print('Esperariamos que cada tamanio fuera siempre el mismo, y que la suma fuera el numero total de datos')
for i in range(nParticiones):
    sizeTrain = len(listaParticiones[i].indicesTrain)
    sizeTest = len(listaParticiones[i].indicesTest)
    print('Particion', i+1, ': ', sizeTest,'+', sizeTrain, '=' , sizeTest+sizeTrain)

    
print('\nAdemas, vamos a comprobar que los conjuntos de test son disjuntos dos a dos, para ver que la particion tiene la estructura deseada:')
for i in range(nParticiones):
    for j in range(i+1, nParticiones):
        set1 = set(listaParticiones[i].indicesTest)
        set2 = set(listaParticiones[j].indicesTest)
        set3 = set1.intersection(set2)
        if(len(set3) == 0):
            print('La interseccion de los conjuntos de test', i+1, 'y', j+1, 'es vacia')


Contamos con  1000 datos.
La lista de particiones de la estrategia tiene longitud  6

Para cada partición, vamos a imprimir el tamanio de los conjuntos de test y entrenamiento, y su suma.
Esperariamos que cada tamanio fuera siempre el mismo, y que la suma fuera el numero total de datos
Particion 1 :  166 + 834 = 1000
Particion 2 :  166 + 834 = 1000
Particion 3 :  166 + 834 = 1000
Particion 4 :  166 + 834 = 1000
Particion 5 :  166 + 834 = 1000
Particion 6 :  166 + 834 = 1000

Ademas, vamos a comprobar que los conjuntos de test son disjuntos dos a dos, para ver que la particion tiene la estructura deseada:
La interseccion de los conjuntos de test 1 y 2 es vacia
La interseccion de los conjuntos de test 1 y 3 es vacia
La interseccion de los conjuntos de test 1 y 4 es vacia
La interseccion de los conjuntos de test 1 y 5 es vacia
La interseccion de los conjuntos de test 1 y 6 es vacia
La interseccion de los conjuntos de test 2 y 3 es vacia
La interseccion de los conjuntos de test 2 y 4 es va

####  Tic-tac-toe
Procedemos de igual manera para el conjunto de datos tic-tac-toe.

In [36]:
nParticiones = 4
dataset = d.Datos('./plantillasArquitectura/datos/tic-tac-toe.data')
estrategia = ep.ValidacionCruzada(nParticiones)
listaParticiones = estrategia.creaParticiones(dataset.datos)

print('Contamos con ', len(dataset.datos), 'datos.')
print('La lista de particiones de la estrategia tiene longitud ', len(listaParticiones))
print('\nPara cada partición, vamos a imprimir el tamanio de los conjuntos de test y entrenamiento, y su suma.')
print('Esperariamos que cada tamanio fuera siempre el mismo, y que la suma fuera el numero total de datos')
for i in range(nParticiones):
    sizeTrain = len(listaParticiones[i].indicesTrain)
    sizeTest = len(listaParticiones[i].indicesTest)
    print('Particion', i+1, ': ', sizeTest,'+', sizeTrain, '=' , sizeTest+sizeTrain)

    
print('\nAdemas, vamos a comprobar que los conjuntos de test son disjuntos dos a dos, para ver que la particion tiene la estructura deseada:')
for i in range(nParticiones):
    for j in range(i+1, nParticiones):
        set1 = set(listaParticiones[i].indicesTest)
        set2 = set(listaParticiones[j].indicesTest)
        set3 = set1.intersection(set2)
        if(len(set3) == 0):
            print('La interseccion de los conjuntos de test', i+1, 'y', j+1, 'es vacia')


Contamos con  958 datos.
La lista de particiones de la estrategia tiene longitud  4

Para cada partición, vamos a imprimir el tamanio de los conjuntos de test y entrenamiento, y su suma.
Esperariamos que cada tamanio fuera siempre el mismo, y que la suma fuera el numero total de datos
Particion 1 :  239 + 719 = 958
Particion 2 :  239 + 719 = 958
Particion 3 :  239 + 719 = 958
Particion 4 :  239 + 719 = 958

Ademas, vamos a comprobar que los conjuntos de test son disjuntos dos a dos, para ver que la particion tiene la estructura deseada:
La interseccion de los conjuntos de test 1 y 2 es vacia
La interseccion de los conjuntos de test 1 y 3 es vacia
La interseccion de los conjuntos de test 1 y 4 es vacia
La interseccion de los conjuntos de test 2 y 3 es vacia
La interseccion de los conjuntos de test 2 y 4 es vacia
La interseccion de los conjuntos de test 3 y 4 es vacia


###  Conclusión (ventajas/desventajas)
Tal y como se ha descrito, queda claro que el modelo de validación cruzada es más simple y computacionalmente menos costoso que la cruzada, de hecho, equivale a la estrategia de validación cruzada donde se ha configurado $K = 1$. 

En contraposición, el modelo de validación cruzada es más lento y costoso, ya que los datos son entrenados y evaluados $K$ veces en vez de una, y la complejidad de la validación crece linealmente con $K$ ya que, aunque el tamaño de las particiones disminuye, aumenta el número de entrenamientos a realizar.

La principal ventaja de la validación cruzada es a su vez una desventaja de la simple. Cuando entrenamos $K$ veces y probamos sobre $K$ conjuntos distintos, conseguimos que el error del modelo devuelto por la validación sea más fiable. Evita que si, para un conjunto de test particular el modelo se ajusta muy, muy bien o muy, muy mal por pura casualidad, caigamos en el error que nuestro modelo es realmente muy bueno o muy malo. Al hacer la media de $K$ errores evitamos resultados de este estilo, ya que si tus $K$ conjuntos de test distintos que se ajustan fatal al modelo, es bastante improbable que esto sea casualidad, y seguramente ocurra que tu modelo es malo.

Este pro de la cruzada explica precisamente la desventaja de la vaidación simple, donde al realizar un sólo test con un sólo conjunto de entrenamiento, no sabemos cómo de fiel es el error obtenido a la realidad


##  Naive-Bayes
___
A continuación, realizaremos los tests necesarios para analizar nuestra implementación de Naive-Bayes.

Extraemos información acerca de las tasas de error según los datos (lenses, german y toc-tac-toe), la estrategia de partición (cruzada o simple) aplicando diferentes parámetros (variamos $K$ y porcentajes) y si ha sido aplicada o no la corrección de Laplace. Cada prueba se realiza 15 veces para extraer luego media y varianza de la tasa de error obtenida.

Además, vamos a separar la ejecución de german.data: debido a los altos tiempos de espera, no vamos a ejecutar validación cruzada con $K$ = 20.

In [8]:
from plantillasArquitectura import Datos as d
from plantillasArquitectura import EstrategiaParticionado as ep
from plantillasArquitectura import Clasificador as cl
import numpy as np
nRepeticiones = 15

dicc = dict()
dicc['LENTILLAS'] = './plantillasArquitectura/datos/conjunto_datos_lentillas.txt'
dicc['TIC-TAC-TOE'] = './plantillasArquitectura/datos/tic-tac-toe.data'

for file in dicc:
    dataset = d.Datos(dicc[file])
    clasificador = cl.ClasificadorNaiveBayes()
    print('\n', file)
    for j in range(2,4):
        porcentaje = 10*j
        estrategia = ep.ValidacionSimple(porcentaje)    
        boolean = True
        for i in range(2):
            errores = np.array([clasificador.validacion(estrategia,dataset,clasificador, laplace = boolean) for k in range(nRepeticiones)])
            mean, std = np.mean(errores), np.std(errores)
            print('\nv. simple,',porcentaje,'%, Laplace a ',boolean,'\nError medio:', mean, '\nDesviacion tipica:', std)
            boolean = False

    for i in range(5,25,5):
        boolean = True
        estrategia = ep.ValidacionCruzada(i)
        for j in range(2):
            errores = np.array([clasificador.validacion(estrategia,dataset,clasificador, laplace = boolean) for k in range(nRepeticiones)])
            mean, std = np.mean(errores), np.std(errores)
            print('\nv.  cruzada, K =',i,', Laplace a ',boolean,'\nError medio:', mean, '\nDesviacion tipica:', std)
            boolean = False




 LENTILLAS

v. simple, 20 %, Laplace a  True 
Error medio: 0.3078620916120916 
Desviacion tipica: 0.06288431756892203

v. simple, 20 %, Laplace a  False 
Error medio: 0.27875956231340593 
Desviacion tipica: 0.010025629775761455

v. simple, 30 %, Laplace a  True 
Error medio: 0.19270586556300842 
Desviacion tipica: 0.07915396342924405

v. simple, 30 %, Laplace a  False 
Error medio: 0.29501812636387975 
Desviacion tipica: 0.0051280339073755685

v.  cruzada, K = 5 , Laplace a  True 
Error medio: 0.3442537185037185 
Desviacion tipica: 0.0355484685450752

v.  cruzada, K = 5 , Laplace a  False 
Error medio: 0.3183124598009054 
Desviacion tipica: 0.003029047492682565

v.  cruzada, K = 10 , Laplace a  True 
Error medio: 0.3245936655936656 
Desviacion tipica: 0.03801324246898937

v.  cruzada, K = 10 , Laplace a  False 
Error medio: 0.3006856976149828 
Desviacion tipica: 0.004700692364615092

v.  cruzada, K = 15 , Laplace a  True 
Error medio: 0.33028935262268594 
Desviacion tipica: 0.00675542

In [10]:
from plantillasArquitectura import Datos as d
from plantillasArquitectura import EstrategiaParticionado as ep
from plantillasArquitectura import Clasificador as cl
import numpy as np
nRepeticiones = 15

dicc = dict()
dicc['GERMAN'] = './plantillasArquitectura/datos/german.data'
file = 'GERMAN'
dataset = d.Datos(dicc[file])
clasificador = cl.ClasificadorNaiveBayes()
print('\n', file)
for j in range(2,4):
    porcentaje = 10*j
    estrategia = ep.ValidacionSimple(porcentaje)    
    boolean = True
    for i in range(2):
        errores = np.array([clasificador.validacion(estrategia,dataset,clasificador, laplace = boolean) for k in range(nRepeticiones)])
        mean, std = np.mean(errores), np.std(errores)
        print('\nv. simple,',porcentaje,'%, Laplace a ',boolean,'\nError medio:', mean, '\nDesviacion tipica:', std)
        boolean = False

for i in range(5,20,5):
    boolean = True
    estrategia = ep.ValidacionCruzada(i)
    for j in range(2):
        errores = np.array([clasificador.validacion(estrategia,dataset,clasificador, laplace = boolean) for k in range(nRepeticiones)])
        mean, std = np.mean(errores), np.std(errores)
        print('\nv.  cruzada, K =',i,', Laplace a ',boolean,'\nError medio:', mean, '\nDesviacion tipica:', std)
        boolean = False



 GERMAN

v. simple, 20 %, Laplace a  True 
Error medio: 0.23460110722610727 
Desviacion tipica: 0.00952263697816008

v. simple, 20 %, Laplace a  False 
Error medio: 0.2446694820323945 
Desviacion tipica: 0.0011478083273373182

v. simple, 30 %, Laplace a  True 
Error medio: 0.2576847664680998 
Desviacion tipica: 0.004215858765510572

v. simple, 30 %, Laplace a  False 
Error medio: 0.2520141059434448 
Desviacion tipica: 0.0013129895377858606

v.  cruzada, K = 5 , Laplace a  True 
Error medio: 0.2518985492285494 
Desviacion tipica: 0.000473868368093443

v.  cruzada, K = 5 , Laplace a  False 
Error medio: 0.2513488633324174 
Desviacion tipica: 0.00022937065414628272

v.  cruzada, K = 10 , Laplace a  True 
Error medio: 0.2507002980352981 
Desviacion tipica: 0.0009652189360731041

v.  cruzada, K = 10 , Laplace a  False 
Error medio: 0.24892872448140674 
Desviacion tipica: 0.0002781312088348454

v.  cruzada, K = 15 , Laplace a  True 
Error medio: 0.24817009907413942 
Desviacion tipica: 0.002

<style type="text/css">
.tg  {border-collapse:collapse;border-spacing:0;border-color:#bbb;}
.tg td{font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;border-color:#bbb;color:#594F4F;background-color:#E0FFEB;}
.tg th{font-family:Arial, sans-serif;font-size:14px;font-weight:normal;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;border-color:#bbb;color:#493F3F;background-color:#9DE0AD;}
.tg .tg-canc{background-color:#91ffb5;font-family:Georgia, serif !important;;border-color:#bbbbbb;text-align:left;vertical-align:top}
.tg .tg-1puz{font-family:Georgia, serif !important;;border-color:#bbbbbb;text-align:center;vertical-align:top}
.tg .tg-mlwj{font-family:Georgia, serif !important;;background-color:#91ffb5;border-color:#bbbbbb;text-align:center;vertical-align:top}
.tg .tg-ytq2{font-family:Georgia, serif !important;;background-color:#ffffff;border-color:#ffffff;text-align:left;vertical-align:top}
.tg .tg-wuod{background-color:#ffffff;font-family:Georgia, serif !important;;border-color:#ffffff;text-align:left;vertical-align:top}
.tg .tg-kzty{background-color:#91ffb5;font-family:Georgia, serif !important;;border-color:#bbbbbb;text-align:center;vertical-align:top}
.tg .tg-n6tj{font-family:Georgia, serif !important;;border-color:#bbbbbb;text-align:left;vertical-align:top}
.tg .tg-5dmk{background-color:#C2FFD6;font-family:Georgia, serif !important;;border-color:#bbbbbb;text-align:center;vertical-align:top}
.tg .tg-2zce{background-color:#C2FFD6;font-family:Georgia, serif !important;;border-color:#bbbbbb;text-align:left;vertical-align:top}
</style>
<table class="tg">
  <tr>
    <th class="tg-ytq2"></th>
    <th class="tg-1puz" rowspan="2"><br>Corrección <br>de Laplace</th>
    <th class="tg-1puz" colspan="2">(mean, std)<br>Error<br>Validación<br>Simple</th>
    <th class="tg-1puz" colspan="4">(mean, std)<br>Error<br>Validación <br>Cruzada</th>
  </tr>
  <tr>
    <td class="tg-wuod"></td>
    <td class="tg-kzty">20% </td>
    <td class="tg-canc">30%</td>
    <td class="tg-canc">K = 5</td>
    <td class="tg-canc">K = 10</td>
    <td class="tg-canc">K = 15</td>
    <td class="tg-canc">K = 20</td>
  </tr>
  <tr>
    <td class="tg-mlwj" rowspan="2"><br>Lenses</td>
    <td class="tg-1puz">Sí</td>
    <td class="tg-n6tj">(0.3, 0.06)</td>
    <td class="tg-n6tj">(0.19, 0.07)</td>
    <td class="tg-n6tj">(0.34, 0.03)</td>
    <td class="tg-n6tj">(0.32, 0.03)</td>
    <td class="tg-n6tj">(0.33, 0.006)</td>
    <td class="tg-n6tj">(0.3, 0.02)</td>
  </tr>
  <tr>
    <td class="tg-5dmk">No</td>
    <td class="tg-2zce">(0.27, 0.01)</td>
    <td class="tg-2zce">(0.29, 0.005)</td>
    <td class="tg-2zce">(0.31, 0.003)</td>
    <td class="tg-2zce">(0.3, 0.004)</td>
    <td class="tg-2zce">(0.3, 0.004)</td>
    <td class="tg-2zce">(0.28, 0.001)</td>
  </tr>
  <tr>
    <td class="tg-mlwj" rowspan="2"><br>Tic-tac-toe</td>
    <td class="tg-1puz">Sí</td>
    <td class="tg-n6tj">(0.3, 0.01)</td>
    <td class="tg-n6tj">(0.29, 0.002)</td>
    <td class="tg-n6tj">(0.30, 8e-4)</td>
    <td class="tg-n6tj">(0.30, 0.001)</td>
    <td class="tg-n6tj">(0.30, 8e-4)</td>
    <td class="tg-n6tj">(0.30, 0.001)</td>
  </tr>
  <tr>
    <td class="tg-5dmk">No</td>
    <td class="tg-2zce">(0.29, 0.001)</td>
    <td class="tg-2zce">(0.29, 0.002)</td>
    <td class="tg-2zce">(0.29, 3e-4)</td>
    <td class="tg-2zce">(0.30, 1e-4)</td>
    <td class="tg-2zce">(0.30, 1e-4)</td>
    <td class="tg-2zce">(0.30, 3e-4)</td>
  </tr>
  <tr>
    <td class="tg-mlwj" rowspan="2"><br>German</td>
    <td class="tg-1puz">Sí</td>
    <td class="tg-n6tj">(0.23, 0.009)</td>
    <td class="tg-n6tj">(0.25, 0.004)</td>
    <td class="tg-n6tj">(0.25, 4e-4)</td>
    <td class="tg-n6tj">(0.25, 9e-4)</td>
    <td class="tg-n6tj">(0.24, 0.002)</td>
    <td class="tg-n6tj"></td>
  </tr>
  <tr>
    <td class="tg-5dmk">No</td>
    <td class="tg-2zce">(0.24, 0.001)</td>
    <td class="tg-2zce">(0.25, 0.001)</td>
    <td class="tg-2zce">(0.25, 2e-4)</td>
    <td class="tg-2zce">(0.24, 2e-4)</td>
    <td class="tg-2zce">(0.24, 3e-4)</td>
    <td class="tg-2zce"></td>
  </tr>
</table>
First row is a table header <style type="text/css">
.tg  {border-collapse:collapse;border-spacing:0;border-color:#bbb;}
.tg td{font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;border-color:#bbb;color:#594F4F;background-color:#E0FFEB;}
.tg th{font-family:Arial, sans-serif;font-size:14px;font-weight:normal;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;border-color:#bbb;color:#493F3F;background-color:#9DE0AD;}
.tg .tg-canc{background-color:#91ffb5;font-family:Georgia, serif !important;;border-color:#bbbbbb;text-align:left;vertical-align:top}
.tg .tg-1puz{font-family:Georgia, serif !important;;border-color:#bbbbbb;text-align:center;vertical-align:top}
.tg .tg-mlwj{font-family:Georgia, serif !important;;background-color:#91ffb5;border-color:#bbbbbb;text-align:center;vertical-align:top}
.tg .tg-ytq2{font-family:Georgia, serif !important;;background-color:#ffffff;border-color:#ffffff;text-align:left;vertical-align:top}
.tg .tg-wuod{background-color:#ffffff;font-family:Georgia, serif !important;;border-color:#ffffff;text-align:left;vertical-align:top}
.tg .tg-kzty{background-color:#91ffb5;font-family:Georgia, serif !important;;border-color:#bbbbbb;text-align:center;vertical-align:top}
.tg .tg-n6tj{font-family:Georgia, serif !important;;border-color:#bbbbbb;text-align:left;vertical-align:top}
.tg .tg-5dmk{background-color:#C2FFD6;font-family:Georgia, serif !important;;border-color:#bbbbbb;text-align:center;vertical-align:top}
.tg .tg-2zce{background-color:#C2FFD6;font-family:Georgia, serif !important;;border-color:#bbbbbb;text-align:left;vertical-align:top}
</style>
<table class="tg">
  <tr>
    <th class="tg-ytq2"></th>
    <th class="tg-1puz" rowspan="2"><br>Corrección <br>de Laplace</th>
    <th class="tg-1puz" colspan="2">(mean, std)<br>Error<br>Validación<br>Simple</th>
    <th class="tg-1puz" colspan="4">(mean, std)<br>Error<br>Validación <br>Cruzada</th>
  </tr>
  <tr>
    <td class="tg-wuod"></td>
    <td class="tg-kzty">20% </td>
    <td class="tg-canc">30%</td>
    <td class="tg-canc">K = 5</td>
    <td class="tg-canc">K = 10</td>
    <td class="tg-canc">K = 15</td>
    <td class="tg-canc">K = 20</td>
  </tr>
  <tr>
    <td class="tg-mlwj" rowspan="2"><br>Lenses</td>
    <td class="tg-1puz">Sí</td>
    <td class="tg-n6tj">(0.3, 0.06)</td>
    <td class="tg-n6tj">(0.19, 0.07)</td>
    <td class="tg-n6tj">(0.34, 0.03)</td>
    <td class="tg-n6tj">(0.32, 0.03)</td>
    <td class="tg-n6tj">(0.33, 0.006)</td>
    <td class="tg-n6tj">(0.3, 0.02)</td>
  </tr>
  <tr>
    <td class="tg-5dmk">No</td>
    <td class="tg-2zce">(0.27, 0.01)</td>
    <td class="tg-2zce">(0.29, 0.005)</td>
    <td class="tg-2zce">(0.31, 0.003)</td>
    <td class="tg-2zce">(0.3, 0.004)</td>
    <td class="tg-2zce">(0.3, 0.004)</td>
    <td class="tg-2zce">(0.28, 0.001)</td>
  </tr>
  <tr>
    <td class="tg-mlwj" rowspan="2"><br>Tic-tac-toe</td>
    <td class="tg-1puz">Sí</td>
    <td class="tg-n6tj">(0.3, 0.01)</td>
    <td class="tg-n6tj">(0.29, 0.002)</td>
    <td class="tg-n6tj">(0.30, 8e-4)</td>
    <td class="tg-n6tj">(0.30, 0.001)</td>
    <td class="tg-n6tj">(0.30, 8e-4)</td>
    <td class="tg-n6tj">(0.30, 0.001)</td>
  </tr>
  <tr>
    <td class="tg-5dmk">No</td>
    <td class="tg-2zce">(0.29, 0.001)</td>
    <td class="tg-2zce">(0.29, 0.002)</td>
    <td class="tg-2zce">(0.29, 3e-4)</td>
    <td class="tg-2zce">(0.30, 1e-4)</td>
    <td class="tg-2zce">(0.30, 1e-4)</td>
    <td class="tg-2zce">(0.30, 3e-4)</td>
  </tr>
  <tr>
    <td class="tg-mlwj" rowspan="2"><br>German</td>
    <td class="tg-1puz">Sí</td>
    <td class="tg-n6tj">(0.23, 0.009)</td>
    <td class="tg-n6tj">(0.25, 0.004)</td>
    <td class="tg-n6tj">(0.25, 4e-4)</td>
    <td class="tg-n6tj">(0.25, 9e-4)</td>
    <td class="tg-n6tj">(0.24, 0.002)</td>
    <td class="tg-n6tj"></td>
  </tr>
  <tr>
    <td class="tg-5dmk">No</td>
    <td class="tg-2zce">(0.24, 0.001)</td>
    <td class="tg-2zce">(0.25, 0.001)</td>
    <td class="tg-2zce">(0.25, 2e-4)</td>
    <td class="tg-2zce">(0.24, 2e-4)</td>
    <td class="tg-2zce">(0.24, 3e-4)</td>
    <td class="tg-2zce"></td>
  </tr>
</table>

 
![alt text](https://github.com/luciaasen/FAA/blob/master/p1/tablas/NB_tables.png?raw=true)

## Scikit-Learn
___
Vamos a ejecutar pruebas similares al apartado anterior, esta vez usando las funciones para partición, entrenamiento y validación del módulo Scikit-Learn.  De nuevo, se varían diferentes estrategias y parámetros, y cada aprendizaje es repetido 15 veces antes de calcular una tasa de error media, así como su desviación típica.

Debido a la necesidad de discretizar parámetros (las funciones del módulo no nos permitían mezclar atributos discretos y continuos) y de realizar muchas llamadas a métodos de la librería (en nuestro caso, el número de llamadas era menor porque las funciones estaban 'envueltas' a nuestra conveniencia), el código de estas pruebas ocupaba el doble que las del apartado anterior y no lo adjuntaremos en el cuaderno. 

No obstante, el fichero de pruebas puede ser encontrado en el directorio de la práctica.

In [2]:
import prueba_scikit
prueba_scikit.pruebaScikit()https://github.com/luciaasen/FAA/blob/master/p1/tablas/NB_tables.png?raw=true


 LENTILLAS

v. simple, 20.0 %, Laplace a  1.0 
Error medio: 0.2 
Desviacion tipica: 5.551115123125783e-17

v. simple, 20.0 %, Laplace a  1e-10 
Error medio: 0.2 
Desviacion tipica: 5.551115123125783e-17

v. simple, 30.0 %, Laplace a  1.0 
Error medio: 0.125 
Desviacion tipica: 0.0

v. simple, 30.0 %, Laplace a  1e-10 
Error medio: 0.125 
Desviacion tipica: 0.0

v.  cruzada, K = 5 , Laplace a  1.0 
Error medio: 0.3899999999999999 
Desviacion tipica: 1.1102230246251565e-16

v.  cruzada, K = 5 , Laplace a  1e-10 
Error medio: 0.3899999999999999 
Desviacion tipica: 1.1102230246251565e-16

v.  cruzada, K = 10 , Laplace a  1.0 
Error medio: 0.31666666666666654 
Desviacion tipica: 1.1102230246251565e-16

v.  cruzada, K = 10 , Laplace a  1e-10 
Error medio: 0.31666666666666654 
Desviacion tipica: 1.1102230246251565e-16

v.  cruzada, K = 15 , Laplace a  1.0 
Error medio: 0.29999999999999993 
Desviacion tipica: 1.1102230246251565e-16

v.  cruzada, K = 15 , Laplace a  1e-10 
Error medio: 0.29999